# Molecular Structure Checking using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of **checking** a **molecular structure** before using it as an input for a **Molecular Dynamics** simulation. The workflow uses the **BioExcel Building Blocks library (biobb)**. The particular structure used is the complex formed by the **SARS-CoV-2 Receptor Binding Domain and the human Angiotensin Converting Enzyme 2** (PDB code [6M0J](https://www.rcsb.org/structure/6m0j)). 

**Structure checking** is a key step before setting up a protein system for **simulations**. A number of **common issues** found in structures at **Protein Data Bank** may compromise the success of the **simulation**, or may suggest that longer **equilibration** procedures are necessary.

The **workflow** shows how to:

- Run **basic manipulations on structures** (selection of models, chains, alternative locations
- Detect and fix **amide assignments** and **wrong chiralities**
- Detect and fix **protein backbone** issues (missing fragments, and atoms, capping)
- Detect and fix **missing side-chain atoms**
- **Add hydrogen atoms** according to several criteria
- Detect and classify **atomic clashes**
- Detect possible **disulfide bonds (SS)**

***NOTE***: Some of the workflow steps work with **AMBER force-field nomenclature** (e.g. CYX, HID, HIE), and thus the output structure might be slightly modified to use it with different **force-fields**.  

An implementation of this workflow in a **web-based Graphical User Interface (GUI)** can be found in the https://mmb.irbbarcelona.org/biobb-wfs/ server (see https://mmb.irbbarcelona.org/biobb-wfs/help/create/structure#check).

***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to check and model 3D structures, create mutations or reconstruct missing atoms.
 - [biobb_amber](https://github.com/bioexcel/biobb_amber): Tools to setup and simulate atomistic MD simulations using AMBER MD package.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to perform chemoinformatics on molecular structures.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_structure_checking.git
cd biobb_wf_structure_checking
conda env create -f conda_env/environment.yml
conda activate biobb_wf_structure_checking
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_cmip/notebooks/biobb_wf_structure_checking.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB structure](#fetch)
 3. [PDB structure checking](#PDBchecking)
     1. [Models](#models)
     2. [Chains](#chains)
     3. [Alternative Locations](#altlocs) 
     4. [Disulfide Bridges](#ss)
     5. [Metal Ions](#metals)
     6. [Ligands](#ligands)
     7. [Hydrogen Atoms](#hydrogens)
     8. [Water Molecules](#water)
     9. [Amide Groups](#amide)
     10. [Chirality](#chirality)
     11. [Missing Side Chain Atoms](#sidechains)
     12. [Missing Backbone Atoms](#backbone)     
     13. [Atomic Clashes](#clashes)
     14. [Final Check](#finalcheck)
 8. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

In [1]:
# TO BE REMOVED!!!

%load_ext autoreload
%autoreload 2

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 6M0J)

In [2]:
import nglview
import ipywidgets

pdbCode = "6m0j" # Should be replaced to check different structures 
#pdbCode = "4wkq" 

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [3]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode,
    'filter': ['ATOM', 'HETATM'],
    'api_id' : 'mmb'
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2022-08-19 13:36:42,878 [MainThread  ] [INFO ]  Downloading: 6m0j from: http://mmb.irbbarcelona.org/api/pdb/6m0j/coords/?
2022-08-19 13:36:43,105 [MainThread  ] [INFO ]  Writting pdb to: 6m0j.pdb
2022-08-19 13:36:43,107 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**: 

In [4]:
# Show original structure
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='spacefill', selection='ion')
view.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

<a id="PDBchecking"></a>
***
## PDB structure checking
The first step of the workflow is a complete **checking** of the **molecular structure**. The **BioBB** utility **structure_check** parses the whole **PDB file**, runs a complete **structure checking**, saves **useful information** about the structure and looks for **possible issues**. The block is printing a **report** of the information extracted from the **input structure**. 
***
**Building Blocks** used:
 - [structure_check](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.structure_check) from **biobb_structure_utils.utils.structure_check**
***

In [5]:
from biobb_structure_utils.utils.structure_check import structure_check

report = pdbCode + ".report.json"

structure_check(
    input_structure_path=downloaded_pdb,
    output_summary_path=report
)

2022-08-19 13:36:43,206 [MainThread  ] [INFO ]  No features provided, all features will be computed: models, chains, altloc, metals, ligands, chiral, getss, cistransbck, backbone, amide, clashes
2022-08-19 13:36:43,207 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:36:43,207 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb --json 6m0j.report.json --check_only --non_interactive checkall

2022-08-19 13:36:54,366 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:36:54,367 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb loaded
 Title: 
 E

0

<a id="StepByStep"></a>
***
## Structure checking, step by step
The **workflow** will go through the **report**, exploring all the different **features** analysed. The following cell is showing the **full content** of the **report**.
***

In [6]:
import json
with open(report, 'r') as f:
  data = json.load(f)
print(json.dumps(data, indent=2))

{
  "add_hydrogen": {
    "detected": {},
    "ionic_detected": [
      "GLU A22",
      "GLU A23",
      "LYS A26",
      "ASP A30",
      "LYS A31",
      "HIS A34",
      "GLU A35",
      "GLU A37",
      "ASP A38",
      "TYR A41",
      "TYR A50",
      "GLU A56",
      "GLU A57",
      "ASP A67",
      "LYS A68",
      "LYS A74",
      "GLU A75",
      "TYR A83",
      "GLU A87",
      "LYS A94",
      "GLU A110",
      "ASP A111",
      "LYS A112",
      "LYS A114",
      "ARG A115",
      "TYR A127",
      "LYS A131",
      "CYS A133",
      "ASP A136",
      "GLU A140",
      "CYS A141",
      "GLU A145",
      "GLU A150",
      "ASP A157",
      "TYR A158",
      "GLU A160",
      "ARG A161",
      "GLU A166",
      "ARG A169",
      "GLU A171",
      "LYS A174",
      "ARG A177",
      "TYR A180",
      "GLU A181",
      "GLU A182",
      "TYR A183",
      "LYS A187",
      "GLU A189",
      "ARG A192",
      "HIS A195",
      "TYR A196",
      "GLU A197",
      "ASP A198",


<a id="models"></a>
##  Models
Presence of **models** in the structure. MD simulations require a single structure, although some structures (e.g. biounits) may be defined as a series of models, in such case all of them are usually required.
***
Use **BioBB extract_model** to select the desired **models**. 

 - [extract_model](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_model) from **biobb_structure_utils.utils.extract_model**
***

In [7]:
print(json.dumps(data['models'], indent=2))

{
  "nmodels": 1
}


In [8]:
from biobb_structure_utils.utils.extract_model import extract_model

pdb_models = pdbCode + ".models.pdb"

prop = {
    'models': [ 1 ]
}

extract_model(
    input_structure_path=downloaded_pdb,
    output_structure_path=pdb_models,
    properties=prop
)

2022-08-19 13:36:54,418 [MainThread  ] [INFO ]  Creating 2b6ed5d5-3214-4f46-806a-663d00f83565 temporary folder
2022-08-19 13:36:54,419 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:36:54,419 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb -o 2b6ed5d5-3214-4f46-806a-663d00f83565/model1.pdb --force_save models --select 1

2022-08-19 13:36:54,882 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:36:54,883 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1

0

In [9]:
# Show modified structure
view = nglview.show_structure_file(pdb_models)
view.add_representation(repr_type='spacefill', selection='ion')
view.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

<a id="chains"></a>
##  Chains
Presence of **chains** in the structure. MD simulations are usually performed with complete structures. However input structure may contain **several copies of the system**, or contain **additional chains** like **peptides** or **nucleic acids** that may be removed. 
***
Use **BioBB extract_chain** to select the desired **chains**. 

 - [extract_chain](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_chain) from **biobb_structure_utils.utils.extract_chain**
***

In [10]:
print(json.dumps(data['chains'], indent=2))

{
  "A": "Protein",
  "E": "Protein"
}


In [11]:
from biobb_structure_utils.utils.extract_chain import extract_chain

pdb_chains = pdbCode + ".chains.pdb"

prop = {
    'chains': [ 'A', 'E' ]
}

extract_chain(
    input_structure_path=pdb_models,
    output_structure_path=pdb_chains,
    properties=prop
)

2022-08-19 13:36:54,977 [MainThread  ] [INFO ]  Selected Chains: A,E
2022-08-19 13:36:54,978 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:36:54,978 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.models.pdb -o 6m0j.chains.pdb --force_save chains --select A,E

2022-08-19 13:36:55,544 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:36:55,545 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.models.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues: 

0

In [12]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_models)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_chains)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="altlocs"></a>
##  Alternative Locations
Presence of residues with **alternative locations**. Atoms with **alternative coordinates** and their **occupancy** are reported. MD simulations requires a **single position** for each atom. 
***
Use **BioBB fix_altloc** to select the desired **alternative locations**. 

 - [fix_altlocs](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_altlocs) from **biobb_model.model.fix_altlocs**
***

In [13]:
print(json.dumps(data['altloc'], indent=2))

{
  "GLN E493": {
    "CA": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "CB": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "CD": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "CG": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "NE2": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "OE1": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ]
  },
  "HIS A228": {
    "CA": [
      {
        "loc_lab

In [14]:
from biobb_model.model.fix_altlocs import fix_altlocs

pdb_altloc = pdbCode + ".altloc.pdb"

prop = {  
    'altlocs': ['E493:A', 'A228:A'],
    'restart': False 
}

fix_altlocs(
    input_pdb_path=pdb_chains,
    output_pdb_path=pdb_altloc,
    properties=prop
)

2022-08-19 13:36:55,682 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:36:55,683 [MainThread  ] [INFO ]  check_structure -i 6m0j.chains.pdb -o 6m0j.altloc.pdb --non_interactive altloc --select E493:A,A228:A

2022-08-19 13:36:56,169 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:36:56,170 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.chains.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  878
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  87
 Num. ligands or modified residues:  7
 Num. water mol.:  80
 Num. atoms:  6558
Metal/Ion residues found
 ZN A901
 CL A902
Small mol ligands found
NAG A903
NAG 

0

In [15]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_chains)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1.add_representation(repr_type='ball+stick', color='red', radius='0.4', selection='228%A 493%A')
view1.add_representation(repr_type='ball+stick', color='green', radius='0.4', selection='228%B 493%B')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_altloc)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2.add_representation(repr_type='ball+stick', radius='0.4', selection='228:A 493:E')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="ss"></a>
##  Disulfide Bridges
Presence of **disulfide bridges** (-S-S- bonds) based on distance criteria. **Disulfide bonds** are crucial for the **structure stability**, and MD simulations require those bonds to be correctly set. Most of the **MD packages** come with tools able to automatically identify -S-S- bonds and include them in the **system topology** parameters, but some of them need the residues involved in the bond to be **explicitly marked** (e.g. AMBER CYX residues). 
***
Use **BioBB fix_ss_bonds** to mark the **disulfide bridges**. 

 - [fix_ssbonds](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_ssbonds) from **biobb_model.model.fix_ssbonds**
***

In [16]:
print(json.dumps(data['getss'], indent=2))

{
  "found": [
    {
      "at1": "CYS A133.SG",
      "at2": "CYS A141.SG",
      "dist": 4.237
    },
    {
      "at1": "CYS A344.SG",
      "at2": "CYS A361.SG",
      "dist": 4.1587
    },
    {
      "at1": "CYS A530.SG",
      "at2": "CYS A542.SG",
      "dist": 4.0947
    },
    {
      "at1": "CYS E336.SG",
      "at2": "CYS E361.SG",
      "dist": 4.1517
    },
    {
      "at1": "CYS E379.SG",
      "at2": "CYS E432.SG",
      "dist": 4.177
    },
    {
      "at1": "CYS E391.SG",
      "at2": "CYS E525.SG",
      "dist": 4.1912
    },
    {
      "at1": "CYS E480.SG",
      "at2": "CYS E488.SG",
      "dist": 4.2686
    }
  ]
}


In [17]:
from biobb_model.model.fix_ssbonds import fix_ssbonds

pdb_ssbonds = pdbCode + ".ssbonds.pdb"

fix_ssbonds(
    input_pdb_path=pdb_altloc,
    output_pdb_path=pdb_ssbonds
)

2022-08-19 13:36:56,311 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:36:56,312 [MainThread  ] [INFO ]  check_structure -i 6m0j.altloc.pdb -o 6m0j.ssbonds.pdb --non_interactive getss --mark All

2022-08-19 13:36:56,887 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:36:56,888 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.altloc.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  878
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  87
 Num. ligands or modified residues:  7
 Num. water mol.:  80
 Num. atoms:  6558
Metal/Ion residues found
 ZN A901
 CL A902
Small mol ligands found
NAG A903
NAG A904
NAG A90

0

In [18]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_altloc)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1.add_representation(repr_type='ball+stick', radius='0.4', selection='CYS')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_ssbonds)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2.add_representation(repr_type='ball+stick', radius='0.2', selection='CYS')
view2.add_representation(repr_type='ball+stick', radius='0.4', selection='CYX')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="metals"></a>
##  Metal Ions
Presence of heteroatoms being **metal ions**. Only structural **metal ions** should be kept in MD simulations, as they require special **force field parameters**. In some cases, the **atomic coordination** between the **protein atoms** and the **metal ions** should be also specified. 
***
Use **BioBB remove_molecules** to remove the undesired **metal ions**. 

 - [remove_molecules](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_molecules) from **biobb_structure-utils.utils.remove_molecules**
***

In [19]:
print(json.dumps(data['metals'], indent=2))

{
  "detected": [
    " ZN A901"
  ]
}


In [20]:
from biobb_structure_utils.utils.remove_molecules import remove_molecules

pdb_metals = pdbCode + ".metals.pdb"

prop = {
    'molecules': [
        {
            'name': 'ZN',
            'res_id': '901',
            'chain': 'A'
        }
    ]
}
remove_molecules(input_structure_path=pdb_ssbonds,
                 output_molecules_path=pdb_metals,
                 properties=prop)


2022-08-19 13:36:57,368 [MainThread  ] [INFO ]  Removed: []


0

In [21]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_chains)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1.add_representation(repr_type='ball+stick', radius='0.4', selection='CYS')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_metals)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2.add_representation(repr_type='ball+stick', radius='0.2', selection='CYS')
view2.add_representation(repr_type='ball+stick', radius='0.4', selection='CYX')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="ligands"></a>
##  Ligands
Presence of heteroatoms being **ligands**. Only important **ligands** should be kept in MD simulations, as they require special **force field parameters**. 
***
Use **BioBB remove_molecules** to remove the undesired **ligands**. 

 - [remove_molecules](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_molecules) from **biobb_structure-utils.utils.remove_molecules**
***

In [22]:
print(json.dumps(data['ligands'], indent=2))

{
  "detected": [
    " ZN A901",
    " CL A902",
    "NAG A903",
    "NAG A904",
    "NAG A905",
    "NAG A906",
    "NAG E601"
  ]
}


In [23]:
from biobb_structure_utils.utils.remove_molecules import remove_molecules

pdb_ligands = pdbCode + ".ligands.pdb"

prop = {
    'molecules': [
        {
            'name': 'CL',
            'res_id': '902',
            'chain': 'A'
        },
        {
            'name' : 'NAG'
        }
    ]
}
remove_molecules(input_structure_path=pdb_metals,
                 output_molecules_path=pdb_ligands,
                 properties=prop)


2022-08-19 13:36:57,831 [MainThread  ] [INFO ]  Removed: []


0

In [24]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_metals)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_ligands)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="hydrogens"></a>
##  Hydrogen atoms
Presence of **hydrogen atoms**. MD setup can be done with the original **hydrogen atoms**, however to prevent possible problems coming from non **standard labelling**, removing them is safer. 
***
Use **BioBB reduce_remove_hydrogens** to remove the undesired **hydrogen atoms**. 

 - [reduce_remove_hydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_remove_hydrogens) from **biobb_chemistry.ambertools.reduce_remove_hydrogens**
***

In [25]:
print(json.dumps(data['stats']['num_h'], indent=2))

0


In [27]:
from biobb_chemistry.ambertools.reduce_remove_hydrogens import reduce_remove_hydrogens

pdb_hydrogens = pdbCode + ".hydrogens.pdb"

reduce_remove_hydrogens(
    input_path=pdb_ligands,
    output_path=pdb_hydrogens
)

2022-08-19 13:38:08,353 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:38:08,354 [MainThread  ] [INFO ]  reduce -Trim /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.ligands.pdb > 6m0j.hydrogens.pdb

2022-08-19 13:38:08,448 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:38:08,449 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "/Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.ligands.pdb"
Trimming: removed 0 hydrogens (0 hets)
If you publish work which uses reduce, please cite:
Word, et. al. (1999) J. Mol. Biol. 285, 1735-1747.
For more information see http://kinemage.biochem.duke.edu



0

In [28]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_ligands)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1.add_representation(repr_type='spacefill', selection='hydrogen')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_hydrogens)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2.add_representation(repr_type='spacefill', selection='hydrogen')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="water"></a>
##  Water molecules
Presence of **water molecules**. **Crystallographic water molecules** may be relevant for keeping the structure stable, however in most cases only some of them are required. These can be later added using other methods.
***
Use **BioBB remove_pdb_water** to remove the undesired **water molecules**. 

 - [remove_pdb_water](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_pdb_water) from **biobb_structure-utils.utils.remove_pdb_water**
***

In [29]:
print(json.dumps(data['stats']['num_wat'], indent=2))

80


In [30]:
from biobb_structure_utils.utils.remove_pdb_water import remove_pdb_water

pdb_water = pdbCode + ".water.pdb"

remove_pdb_water(
    input_pdb_path=pdb_hydrogens,
    output_pdb_path=pdb_water
)

2022-08-19 13:38:18,750 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:38:18,750 [MainThread  ] [INFO ]  check_structure -i 6m0j.hydrogens.pdb -o 6m0j.water.pdb --force_save water --remove yes

2022-08-19 13:38:19,316 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:38:19,318 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.hydrogens.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  871
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  80
 Num. ligands or modified residues:  0
 Num. water mol.:  80
 Num. atoms:  6486

Running water. Options: --remove yes
80 Water molecules detected
80 Water molecules remove

0

In [31]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_hydrogens)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.5', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_water)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.5', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="amide"></a>
##  Amide groups
Presence of incorrect **amide groups**. **Amide terminal atoms** in ***Asparagines*** and ***Glutamines*** residues can be labelled incorrectly, with swapped **Nitrogen** and **Oxygen** atoms from the **amide group**. 
***
Use **BioBB fix_amides** to fix the incorrectly assigned **amide groups**. 

 - [fix_amides](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_amides) from **biobb_model.model.fix_amides**
***

In [32]:
print(json.dumps(data['amide'], indent=2))

{
  "detected": [
    {
      "at1": "LYS A31.NZ",
      "at2": "GLN E493.NE2",
      "dist": 2.9256
    },
    {
      "at1": "GLN A42.NE2",
      "at2": "GLN E498.NE2",
      "dist": 2.9272
    },
    {
      "at1": "ASN A103.OD1",
      "at2": "ASN A194.OD1",
      "dist": 2.8068
    },
    {
      "at1": "ASN A134.OD1",
      "at2": "GLU A140.OE2",
      "dist": 2.7852
    },
    {
      "at1": "ASN A134.ND2",
      "at2": "ASN A137.N",
      "dist": 3.0824
    },
    {
      "at1": "GLU A150.O",
      "at2": "ASN A154.OD1",
      "dist": 2.8946
    },
    {
      "at1": "ARG E357.NH1",
      "at2": "ASN E394.ND2",
      "dist": 2.9629
    }
  ],
  "n_amides": 92
}


In [33]:
from biobb_model.model.fix_amides import fix_amides

pdb_amides = pdbCode + ".amides.pdb"

fix_amides(
    input_pdb_path=pdb_water,
    output_pdb_path=pdb_amides
)

2022-08-19 13:38:24,367 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:38:24,368 [MainThread  ] [INFO ]  check_structure -i 6m0j.water.pdb -o 6m0j.amides.pdb --force_save amide --fix All

2022-08-19 13:38:25,996 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:38:25,997 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.water.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  6406

Running amide. Options: --fix All
7 unusual contact(s) involving amide atoms found
 LYS A31.NZ   GLN E

0

In [34]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_water)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1.add_representation(repr_type='ball+stick', radius='0.4', selection='31:A 42:A 103:A 134:A 137:A 140:A 150:A 154:A 194:A 357:E 394:E 493:E 498:E')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_amides)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2.add_representation(repr_type='ball+stick', radius='0.4', selection='31:A 42:A 103:A 134:A 137:A 140:A 150:A 154:A 194:A 357:E 394:E 493:E 498:E')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="chirality"></a>
##  Chirality
Presence of incorrect **chiralities**. Side chains of ***Threonine*** and ***Isoleucine*** residues are **chiral**. <br> **Incorrect atom labelling** can lead to the **wrong chirality**. 
***
Use **BioBB fix_chirality** to fix the incorrectly assigned **chirality**. 

 - [fix_chirality](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_chirality) from **biobb_model.model.fix_chirality**
***

In [35]:
print(json.dumps(data['chiral'], indent=2))

{
  "n_chirals": 72
}


In [36]:
from biobb_model.model.fix_chirality import fix_chirality

pdb_chiral = pdbCode + ".chiral.pdb"

fix_chirality(
    input_pdb_path=pdb_amides,
    output_pdb_path=pdb_chiral
)

2022-08-19 13:38:31,265 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:38:31,266 [MainThread  ] [INFO ]  check_structure -i 6m0j.amides.pdb -o 6m0j.chiral.pdb --force_save chiral --fix All

2022-08-19 13:38:31,725 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:38:31,726 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.amides.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  6406

Running chiral. Options: --fix All
No residues with incorrect side-chain chirality found
Structure 

0

In [38]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_amides)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_chiral)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="sidechains"></a>
##  Side Chains
Presence of **missing protein side chain atoms**. MD programs only work with complete residues; some of the MD packages come with tools able to **model missing side chain atoms**, but not all of them. 
***
Use **BioBB fix_side_chain** to model the **missing side chain atoms**. 

 - [fix_side_chain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [39]:
print(json.dumps(data['fixside'], indent=2))

{}


In [40]:
from biobb_model.model.fix_side_chain import fix_side_chain

pdb_side_chains = pdbCode + ".sidechains.pdb"

fix_side_chain(
    input_pdb_path=pdb_chiral,
    output_pdb_path=pdb_side_chains
)

2022-08-19 13:38:37,798 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:38:37,799 [MainThread  ] [INFO ]  check_structure -i 6m0j.chiral.pdb -o 6m0j.sidechains.pdb --force_save fixside --fix ALL

2022-08-19 13:38:38,257 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:38:38,258 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.chiral.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  6406

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found

0

In [41]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_chiral)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_side_chains)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="backbone"></a>
##  Backbone
Presence of **missing backbone atoms**. **PDB files** can have **missing backbone atoms**, usually placed in **flexible regions** of the molecule. **MD programs** work with amino acid libraries that need all atoms to be present. Some MD packages tools are able to model **side chain missing atoms**, but they are rarely capable of modeling **backbone atoms**. 
***
Use **BioBB fix_backbone** to model the **missing backbone atoms**. <br>
***NOTE:*** *This building block is using the Modeler tool, which requires an academic license.*

 - [fix_backbone](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_backbone) from **biobb_model.model.fix_backbone**
 
An additional **building block** is used to extract the **canonical FASTA sequence** for the protein, to be used in **Modeler** to model the **missing backbone region**:

 - [canonical_fasta](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.canonical_fasta) from **biobb_io.api.canonical_fasta**

***

In [42]:
print(json.dumps(data['backbone'], indent=2))

{
  "missing_atoms": {
    "ASP A615": [
      "OXT"
    ],
    "GLY E526": [
      "OXT"
    ]
  }
}


In [43]:
from biobb_io.api.canonical_fasta import canonical_fasta

pdb_fasta = pdbCode + ".fasta"

prop = {
    'pdb_code': pdbCode,
    'api_id': 'pdb'
}

canonical_fasta(
    output_fasta_path=pdb_fasta,
    properties=prop
)

2022-08-19 13:38:41,953 [MainThread  ] [INFO ]  Downloading: 6m0j from: https://www.rcsb.org/fasta/entry/6m0j
2022-08-19 13:38:42,405 [MainThread  ] [INFO ]  Writting FASTA to: 6m0j.fasta


/opt/anaconda3/envs/biobb_wf_structure_checking/lib/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.rcsb.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0

In [44]:
from biobb_model.model.fix_backbone import fix_backbone

pdb_backbone = pdbCode + ".backbone.pdb"

fix_backbone(
    input_pdb_path=pdb_side_chains,
    input_fasta_canonical_sequence_path=pdb_fasta,
    output_pdb_path=pdb_backbone
)

2022-08-19 13:38:42,449 [MainThread  ] [INFO ]  Modeller is not installed in your environment.
Please install it by typing:

conda install -c salilab modeller

2022-08-19 13:38:42,450 [MainThread  ] [INFO ]  Modeller is not installed, the execution of this block will be interrupted


1

In [45]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_side_chains)
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_backbone)
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

ValueError: you must provide file extension if using file-like object or text content

<a id="clashes"></a>
##  Atomic Clashes
Presence of **atomic clashes**. Atoms that are **too close in space** can have a problem of **energetic repulsion**. Most clashes come from **over-compactation** of crystal structures and are naturally corrected on **system setup** or **MD equilibration**, but may lead to a significant **distortion** of the structure. **Clashes** are detected based on **distance criteria** and are classified in different groups, depending on the **atom types** involved:

- **Severe**: Atoms too close, usually indicating superimposed structures or badly modelled regions. Should be fixed.
- **Apolar**: Vdw collisions. Usually fixed during the simulation.
- **Polar and ionic**: Usually indicate wrong side chain conformations. Usually fixed during the simulation

***
Use the **BioBB_amber** module to energetically minimize the structure and fix **atomic clashes**.

 - [leap_gen_top](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_gen_top) from **biobb_amber.leap.leap_gen_top**
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [amber_to_pdb](https://biobb-amber.readthedocs.io/en/latest/ambpdb.html#module-ambpdb.amber_to_pdb) from **biobb_amber.ambpdb.amber_to_pdb**

***

In [ ]:
print(json.dumps(data['clashes'], indent=2))

In [46]:
from biobb_amber.leap.leap_gen_top import leap_gen_top

pdb_amber = pdbCode + ".amber.pdb"
top_amber = pdbCode + ".amber.top"
crd_amber = pdbCode + ".amber.crd"

prop = {
    'forcefield': ['protein.ff14SB']
}

## TO BE REMOVED!!!
pdb_backbone = pdb_side_chains

leap_gen_top(
    input_pdb_path=pdb_backbone,
    output_pdb_path=pdb_amber,
    output_top_path=top_amber,
    output_crd_path=crd_amber,
    properties=prop
)

2022-08-19 13:42:34,006 [MainThread  ] [INFO ]  Creating 3caec99d-4d99-4b15-9a51-9ebafbac65d2 temporary folder
2022-08-19 13:42:34,007 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:42:34,008 [MainThread  ] [INFO ]  tleap  -f 3caec99d-4d99-4b15-9a51-9ebafbac65d2/leap.in

2022-08-19 13:42:37,149 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:42:37,150 [MainThread  ] [INFO ]  -I: Adding /opt/anaconda3/envs/biobb_wf_structure_checking/dat/leap/prep to search path.
-I: Adding /opt/anaconda3/envs/biobb_wf_structure_checking/dat/leap/lib to search path.
-I: Adding /opt/anaconda3/envs/biobb_wf_structure_checking/dat/leap/parm to search path.
-I: Adding /opt/anaconda3/envs/biobb_wf_structure_checking/dat/leap/cmd to search path.
-f: Source 3caec99d-4d99-4b15-9a51-9ebafbac65d2/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./3caec99d-4d99-4b15-9a51-9ebafbac65d2/leap.in
----- Source: /opt/anaconda3/envs/biobb_wf_structure_checking/dat/leap/cmd/leaprc.protein.f

0

In [47]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

trj_amber = pdbCode + ".amber.crd"
rst_amber = pdbCode + ".amber.rst"
log_amber = pdbCode + ".amber.log"

prop = {
    'simulation_type' : 'minimization',
    'mdin' : {
        'ntb' : 0,        # Periodic Boundary. No periodicity is applied and PME (Particle Mesh Ewald) is off.
        'cut' : 12,       # Nonbonded cutoff, in Angstroms.
        'maxcyc' : 500,   # Maximum number of cycles of minimization.
        'ncyc' : 50       # Minimization will be switched from steepest descent to conjugate gradient after ncyc cycles.
    }
}

sander_mdrun(
     input_top_path=top_amber,
     input_crd_path=crd_amber,
     output_traj_path=trj_amber,
     output_rst_path=rst_amber,
     output_log_path=log_amber,
     properties=prop
)

2022-08-19 13:42:41,413 [MainThread  ] [INFO ]  Creating b8fbc09b-e03c-41c2-bee6-38476d24dc2d temporary folder
2022-08-19 13:42:41,414 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:42:41,415 [MainThread  ] [INFO ]  sander -O -i b8fbc09b-e03c-41c2-bee6-38476d24dc2d/sander.mdin -p 6m0j.amber.top -c 6m0j.amber.crd -r 6m0j.amber.rst -o 6m0j.amber.log -x 6m0j.amber.crd

2022-08-19 13:43:49,325 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:43:49,327 [MainThread  ] [INFO ]  Removed: ['b8fbc09b-e03c-41c2-bee6-38476d24dc2d', 'mdinfo']


0

In [48]:
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb

pdb_amber_min = pdbCode + ".amber-min.pdb"

amber_to_pdb(
      input_top_path=top_amber,
      input_crd_path=rst_amber,
      output_pdb_path=pdb_amber_min
)

2022-08-19 13:43:53,273 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:43:53,274 [MainThread  ] [INFO ]  ambpdb  -p 6m0j.amber.top -c 6m0j.amber.rst >  6m0j.amber-min.pdb

2022-08-19 13:43:54,047 [MainThread  ] [INFO ]  Exit code 0



0

In [52]:
# Show modified structure
view1 = nglview.show_structure_file(pdb_backbone)
view1.add_representation(repr_type='ball+stick', selection='all')
view1.add_representation(repr_type='spacefill', selection='ion')
view1.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view1._remote_call('setSize', target='Widget', args=['','600px'])
view2 = nglview.show_structure_file(pdb_amber_min)
view2.clear_representations()
view2.add_representation(repr_type='cartoon', selection='all')
view2.add_representation(repr_type='ball+stick', selection='all')
view2.add_representation(repr_type='spacefill', selection='ion')
view2.add_representation(repr_type='spacefill', radius='0.3', selection='water')
view2._remote_call('setSize', target='Widget', args=['','600px'])
ipywidgets.HBox([view1, view2])

<a id="finalcheck"></a>
##  Final Check
Checking the **final structure** to analyse the **possible issues** present in the **modified structure** in comparison with the issues we found with the **original structure** at the beginning of the workflow. 

In [53]:
from biobb_structure_utils.utils.structure_check import structure_check

report_final = pdbCode + ".report_final.json"

structure_check(
    input_structure_path=pdb_amber_min,
    output_summary_path=report_final
)

2022-08-19 13:45:31,672 [MainThread  ] [INFO ]  No features provided, all features will be computed: models, chains, altloc, metals, ligands, chiral, getss, cistransbck, backbone, amide, clashes
2022-08-19 13:45:31,673 [MainThread  ] [INFO ]  Not using any container
2022-08-19 13:45:31,674 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.amber-min.pdb --json 6m0j.report_final.json --check_only --non_interactive checkall

2022-08-19 13:45:43,047 [MainThread  ] [INFO ]  Exit code 0

2022-08-19 13:45:43,048 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure /Users/hospital/BioBB/Notebooks_dev/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.amber-

0

In [54]:
import json
with open(report_final, 'r') as f:
  data_final = json.load(f)
print(json.dumps(data_final, indent=2))

{
  "add_hydrogen": {
    "detected": {},
    "ionic_detected": [
      "GLU  4",
      "GLU  5",
      "LYS  8",
      "ASP  12",
      "LYS  13",
      "GLU  17",
      "GLU  19",
      "ASP  20",
      "TYR  23",
      "TYR  32",
      "GLU  38",
      "GLU  39",
      "ASP  49",
      "LYS  50",
      "LYS  56",
      "GLU  57",
      "TYR  65",
      "GLU  69",
      "LYS  76",
      "GLU  92",
      "ASP  93",
      "LYS  94",
      "LYS  96",
      "ARG  97",
      "TYR  109",
      "LYS  113",
      "ASP  118",
      "GLU  122",
      "GLU  127",
      "GLU  132",
      "ASP  139",
      "TYR  140",
      "GLU  142",
      "ARG  143",
      "GLU  148",
      "ARG  151",
      "GLU  153",
      "LYS  156",
      "ARG  159",
      "TYR  162",
      "GLU  163",
      "GLU  164",
      "TYR  165",
      "LYS  169",
      "GLU  171",
      "ARG  174",
      "TYR  178",
      "GLU  179",
      "ASP  180",
      "TYR  181",
      "ASP  183",
      "TYR  184",
      "ARG  186",
      "

In [ ]:
# import jsondiff
# res = jsondiff.diff(data, data_final)
# #res = jsondiff.diff(data, data_final, syntax='explicit')
# if res:
#     print("Diff found")
#     print(res)
#     #print(json.dumps(res, indent=2))
# else:
#     print("Same")

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)